In [ ]:
#q1

In [10]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [30]:
!pip install rouge

In [21]:
import pandas as pd
import tqdm
import numpy as np

In [4]:
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"

In [7]:
url = f'{github_url}?raw=1'
df = pd.read_csv(url)
df = df.iloc[:300]

In [8]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [11]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
answer_llm = df.iloc[0].answer_llm
answer_llm_encoded = embedding_model.encode(answer_llm)

In [16]:
answer_llm_encoded[0]


-0.42244658

In [ ]:
#q2

In [23]:
evaluations = []
for i in tqdm.tqdm(np.arange(df.shape[0])):
  ans_llm = embedding_model.encode(df.iloc[i,0])
  ans_ori = embedding_model.encode(df.iloc[i,1])
  score = ans_llm.dot(ans_ori)
  evaluations.append(score)


100%|██████████| 300/300 [05:02<00:00,  1.01s/it]


In [24]:
np.percentile(evaluations,75)

31.674307823181152

In [ ]:
#q3

In [26]:
def normalize_vector(v):
  norm = np.sqrt((v * v).sum())
  v_norm = v / norm
  return v_norm

In [28]:
evaluations = []
for i in tqdm.tqdm(np.arange(df.shape[0])):
  ans_llm = normalize_vector(embedding_model.encode(df.iloc[i,0]))
  ans_ori = normalize_vector(embedding_model.encode(df.iloc[i,1]))
  score = ans_llm.dot(ans_ori)
  evaluations.append(score)


100%|██████████| 300/300 [04:59<00:00,  1.00it/s]


In [29]:
np.percentile(evaluations,75)

0.8362347930669785

In [ ]:
#q4

In [31]:
from rouge import Rouge
rouge_scorer = Rouge()

In [35]:
df[df.document == "5170565b"]

,answer_llm,answer_orig,document,question,course
10,"Yes, all sessions are recorded, so if you miss...","Everything is recorded, so you won’t miss anyt...",5170565b,Are sessions recorded if I miss one?,machine-learning-zoomcamp
11,"Yes, you can ask your questions in advance if ...","Everything is recorded, so you won’t miss anyt...",5170565b,Can I ask questions in advance if I can't atte...,machine-learning-zoomcamp
12,"If you miss a session, don't worry! Everything...","Everything is recorded, so you won’t miss anyt...",5170565b,How will my questions be addressed if I miss a...,machine-learning-zoomcamp
13,"Yes, there is a way to catch up on a missed se...","Everything is recorded, so you won’t miss anyt...",5170565b,Is there a way to catch up on a missed session?,machine-learning-zoomcamp
14,"Yes, you can still interact with instructors a...","Everything is recorded, so you won’t miss anyt...",5170565b,Can I still interact with instructors after mi...,machine-learning-zoomcamp


In [36]:
r = df[df.index == 10]

In [38]:
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [47]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [48]:
np.mean([scores['rouge-1']["r"],scores["rouge-2"]["r"],scores['rouge-l']["r"]])

0.35490035490035493

In [49]:
#q5
r1 = []
r2 = []
rl = []
for i in tqdm.tqdm(np.arange(df.shape[0])):
  scores = rouge_scorer.get_scores(df.iloc[i,:]['answer_llm'], df.iloc[i,:]['answer_orig'])[0]
  rouge_1 = scores['rouge-1']['f']
  rouge_2 = scores['rouge-2']['f']
  rouge_l = scores['rouge-l']['f']
  r1.append(rouge_1)
  r2.append(rouge_2)
  rl.append(rouge_l)


100%|██████████| 300/300 [00:01<00:00, 230.07it/s]


In [50]:
np.mean(r2)

0.20696501983423318